In [9]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
import cesium

In [10]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/LCS2/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/LCS2/OFF_data/"

In [11]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)     

In [12]:
# Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [13]:


lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Labels = []

nbfeatures=5

a=binning
b=nbfeatures
c=lg

# multivariate

data_matrix= np.zeros((b,a))
on=[]
off=[]
photon_idx=[]
delta_index=[]
flux=[]
delta_flux=[]
mjd=[]
fratio=[]
#Construct data matrix
for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    photon_idx.append(np.array(dataframe['Photon Index']))
    delta_index.append(np.array(dataframe['Delta Index']))
    mjd.append(np.array(dataframe['MJD']))
    flux.append(np.array(dataframe['Flux']))
    delta_flux.append(np.array(dataframe['Delta Flux']))
    
    fratio.append(np.array(dataframe['fratio']))
for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    photon_idx.append(np.array(dataframe['Photon Index']))
    delta_index.append(np.array(dataframe['Delta Index']))
    mjd.append(np.array(dataframe['MJD']))
    flux.append(np.array(dataframe['Flux']))
    fratio.append(np.array(dataframe['fratio']))
#Creating labels
for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    Labels.append(int(1))


In [14]:
import numpy as np
import cesium
from cesium import featurize
import warnings
warnings.filterwarnings("ignore")


features_to_use = [
                  'std','amplitude','max_slope']
# features_to_use = [
#                   "max_slope",
#                   "med_err","stetson_j","std_err"]
fset_cesium = featurize.featurize_time_series(times=mjd,
                                              values=photon_idx,
                                              errors=delta_index,
                                              features_to_use=features_to_use)
print(fset_cesium)

feature       std amplitude max_slope
channel         0         0         0
0        0.836174  1.610176  0.021743
1        1.096913  1.986321  0.029427
2        0.233749  0.468603  0.004678
3        0.262551  0.695831  0.008163
4        0.264472  0.530773  0.006679
...           ...       ...       ...
1084     0.885969  1.844204  0.026987
1085     1.387577  2.500000  0.025040
1086     0.639641  2.078685  0.023031
1087     1.180720  1.932658  0.026015
1088     0.094361  0.203975  0.001825

[1089 rows x 3 columns]


In [37]:
import numpy as np
import cesium
from cesium import featurize
import warnings
warnings.filterwarnings("ignore")


features_to_use = [
                  'std','amplitude','max_slope']
# features_to_use = [
#                   "max_slope",
#                   "med_err","stetson_j","std_err"]
fset_cesium = featurize.featurize_time_series(times=mjd,
                                              values=fratio,
                                            
                                              features_to_use=features_to_use)
print(fset_cesium)

feature       std amplitude max_slope
channel         0         0         0
0        0.095021  0.161529  0.001610
1        0.098448  0.221251  0.002105
2        0.064389  0.139161  0.001374
3        0.076272  0.164554  0.001337
4        0.059814  0.168744  0.001831
...           ...       ...       ...
1084     0.136335  0.297780  0.004256
1085     0.099619  0.153765  0.002003
1086     0.123454  0.401792  0.004205
1087     0.164273  0.288976  0.003062
1088     0.135775  0.284899  0.002006

[1089 rows x 3 columns]


In [7]:
import numpy as np
import cesium
from cesium import featurize
import warnings
warnings.filterwarnings("ignore")


features_to_use = [
                  "max_slope",'std','amplitude','med_err']
# features_to_use = [
#                   "max_slope",
#                   "med_err","stetson_j","std_err"]
fset_cesium = featurize.featurize_time_series(times=mjd,
                                              values=flux,
                                              errors=delta_flux,
                                              features_to_use=features_to_use)
print(fset_cesium)

feature     max_slope           std     amplitude       med_err
channel             0             0             0             0
0        3.090637e-10  1.155174e-08  2.546799e-08  5.599316e-09
1        3.091292e-10  1.336450e-08  2.780036e-08  3.097499e-09
2        3.493579e-10  9.980830e-09  2.722688e-08  5.803230e-09
3        3.916268e-10  2.392468e-08  4.835708e-08  7.057930e-09
4        1.345671e-10  4.980775e-09  1.134703e-08  4.495170e-09
...               ...           ...           ...           ...
1011     7.277061e-10  2.981292e-08  6.196553e-08  1.012394e-08
1012     1.666722e-10  6.278653e-09  1.359814e-08  1.313846e-09
1013     2.643578e-10  1.008355e-08  2.112031e-08  6.434562e-10
1014     1.719634e-10  8.414585e-09  1.306731e-08  1.340521e-09
1015     1.538337e-10  6.592575e-09  1.346074e-08  6.536608e-09

[1016 rows x 4 columns]


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
iterations =10
for i in  range(iterations):
# "mean",
#                    "skew",
#                    
    train, test = train_test_split(np.arange(lg), random_state=i)

    model_cesium = RandomForestClassifier(n_estimators=500, max_features="auto",
                                          random_state=i)
    Labels=np.hstack(Labels)
    model_cesium.fit(fset_cesium.iloc[train], Labels[train])
    prediction= model_cesium.predict(fset_cesium)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(Labels)):
        if Labels[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
            else : 
                fon+=1
        if Labels[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(prediction,Labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))

Accuracy for ON class:  75.75342465753424 %
Accuracy for OFF class:  99.96062992125984 %
False Positive rate:  24.246575342465754 %
False Negative rate:  0.03937007874015748 %
F1 score:  0.9844643890835931


In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

train, test = train_test_split(np.arange(lg), random_state=0)

model_cesium = KNeighborsClassifier(2)
Labels=np.hstack(Labels)
model_cesium.fit(fset_cesium.iloc[train], Labels[train])

KNeighborsClassifier(n_neighbors=2)

In [177]:
from sklearn.metrics import accuracy_score

preds_cesium = model_cesium.predict(fset_cesium)

In [178]:
print("Built-in cesium features: training accuracy={:.2%}, test accuracy={:.2%}".format(
          accuracy_score(preds_cesium[train], Labels[train]),
          accuracy_score(preds_cesium[test], Labels[test])))

Built-in cesium features: training accuracy=91.18%, test accuracy=88.28%


In [179]:
preds_cesium[test]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [180]:
Labels[test]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])